In [7]:
import numpy as np
import pandas as pd

# LU decomposition
def Solve_SE(matA, matb):
    rowA = len(matA)
    colA = len(matA.T)
    rowB = len(matb)
    if (rowA != colA) | (rowB != colA):
        raise NotImplementedError
    else:
        pass
    L = np.zeros((rowA, colA))
    U = np.zeros((rowA, colA))
 
    for j in range(0, len(L.T)):
        for i in range(0, len(L)):
            if j==0:
                L[i, j] = matA[i, j]
                U[j, i] = matA[j, i] / L[0, 0]
            else:
                if i < j:
                    L[i, j] = 0
                else:
                    Lsum = 0
                    for k in range(0, j):
                        Lsum = Lsum + L[i, k]*U[k, j]
                    L[i, j] = matA[i, j] - Lsum
            if i < j:
                U[j, i] = 0
            elif i == j:
                U[j, i] = 1
            else:
                Usum = 0
                for k in range(0, i):
                    Usum = Usum + L[k, i]*U[j, k]
                U[j, i] = (matA[j, i] - Usum) /L[j, j]
 
 
    modB = np.zeros((len(matb), 1))
    modB[0] = matb[0] / L[0, 0]
 
    for i in range(0, len(modB)):
        sumofB = 0
        for k in range(0, i):
            sumofB = sumofB + L[i, k]*modB[k]
        modB[i] = (matb[i] - sumofB) / L[i, i]
 
    x = np.zeros((len(modB), 1))
    for no in range(len(x)-1, 0-1, -1):
        sum=0
        for i in range(0, len(x)):
            sum = sum + U[no, i] * x[i]
        x[no] = modB[no] - sum
    return x

# europian call option 
k=50; r=0.10; t=5/12; sigma=0.4; smax=100; q=0
m=20; n=10
ds = smax/m
dt = t/n
matdim=m-2
 
f = np.zeros((n+1, m+1))
matA = np.zeros((matdim+1, matdim+1))
matb = np.zeros((matdim+1, 1))
A = np.zeros((matdim+1, 1))
B = np.zeros((matdim+1, 1))
C = np.zeros((matdim+1, 1))
 
for j in range(1, m):
    f[n, j] = max(j*ds - k, 0)
for i in range(0, n+1):
    f[i, 0] = 0
for i in range(0, n+1):
    f[i, m] = max(m*ds - k*np.exp(-(r-q)*(n-i)*dt), 0)
 
for j in range(m-1, 1-1, -1):
    A[j-1] = 0.5 * dt * ((r-q) * j - (sigma ** 2) * (j ** 2))
    B[j-1] = 1 + ((sigma * j) ** 2) * dt + r * dt
    C[j-1] = 0.5 * dt * (-(r-q) * j - (sigma ** 2) * (j ** 2))
 
for i in range(n-1, 0-1, -1):
    mm = matdim
    for ii in range(0, matdim+1):
        for jj in range(0, matdim+1):
            if ii == jj +1:
                matA[ii, jj] = C[mm]
            elif ii == jj:
                matA[ii, jj] = B[mm]
            elif ii+1 == jj:
                matA[ii, jj] = A[mm]
            else:
                matA[ii, jj] = 0
        mm=mm-1
 
    matb[0] = f[i+1, m-1] - C[matdim]*f[i,m]
    matb[matdim] = f[i+1, 1] - A[0]*f[i,0]
 
    for jj in range(matdim-1, 1-1, -1):
        matb[jj] = f[i+1, jj+1]
    
    # LU분해가 소수점이 들어간 작은 수일 경우 pivoting을 안해주면 오차가 크게 나는 것으로 보임
    # matx = Solve_SE(matA, matb)
    # numpy 메서드를 사용했음
    matx = np.linalg.solve(matA,matb)
 
    jj=m-1
    for ii in range(len(matx)):
        f[i, jj]=matx[ii]
        jj=jj-1
 
for j in range(m+1):
    print("stock price: ", j*ds, "option price: ", round(f[0, j], 2))


df = pd.DataFrame(index=[x for x in range(n+1)],
                  columns=[y for y in range(m+1)])
for i in range(n, 0-1, -1):
    for j in range(m, 0-1, -1):
        df.loc[i,j] = f[i,j]

df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
name = 'call option'
df.to_excel('implicit FDM of europian option.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

stock price:  0.0 option price:  0.0
stock price:  5.0 option price:  1.59
stock price:  10.0 option price:  30.06
stock price:  15.0 option price:  20.58
stock price:  20.0 option price:  12.96
stock price:  25.0 option price:  7.66
stock price:  30.0 option price:  4.41
stock price:  35.0 option price:  2.78
stock price:  40.0 option price:  2.45
stock price:  45.0 option price:  3.24
stock price:  50.0 option price:  5.08
stock price:  55.0 option price:  7.9
stock price:  60.0 option price:  11.52
stock price:  65.0 option price:  15.69
stock price:  70.0 option price:  20.18
stock price:  75.0 option price:  24.81
stock price:  80.0 option price:  29.51
stock price:  85.0 option price:  34.27
stock price:  90.0 option price:  39.19
stock price:  95.0 option price:  44.73
stock price:  100.0 option price:  52.04


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0,1.58924,30.0613,20.5833,12.9639,7.66188,4.40657,2.78181,2.44981,3.23644,5.07944,7.90051,11.5194,15.6891,20.176,24.8115,29.5103,34.2731,39.1909,44.7329,52.0405
1,0,1.44363,38.2879,33.9238,29.2631,24.5758,19.8968,15.3413,11.1082,7.46408,4.6774,2.9154,2.2138,2.61079,4.29026,7.61239,13.002,20.7202,30.6134,42.286,51.8403
2,0,1.25188,28.0106,18.0875,10.6321,5.82129,3.08906,1.84297,1.70331,2.55631,4.4501,7.38425,11.1601,15.4808,20.0803,24.7751,29.4644,34.1304,38.8616,44.1711,51.6392
3,0,1.11371,38.0223,33.8771,29.2951,24.5892,19.8307,15.1466,10.7427,6.92175,4.02127,2.23445,1.48675,1.69013,2.97333,5.7509,10.6375,18.1975,28.5462,41.2519,51.4373
4,0,0.919824,25.2551,14.9526,7.9857,3.95357,1.89095,1.04862,1.05003,1.88274,3.76446,6.82807,10.8002,15.2964,20.0224,24.7946,29.5011,34.0859,38.5916,43.5505,51.2345
5,0,0.79299,37.908,33.9821,29.4183,24.651,19.7986,14.9809,10.382,6.32884,3.28315,1.55645,0.863165,0.927446,1.7887,3.86941,7.95411,15.0227,25.7637,39.9524,51.0309
6,0,0.596246,21.2127,10.9168,5.0692,2.18987,0.913636,0.457282,0.518854,1.20159,2.95659,6.21629,10.4531,15.1449,19.9979,24.861,29.6295,34.1959,38.4749,42.8613,50.8264
7,0,0.487803,38.1942,34.3021,29.6141,24.7346,19.7911,14.8588,10.055,5.6763,2.36287,0.871374,0.376096,0.38035,0.840714,2.10869,5.00474,10.9298,21.6703,38.2444,50.6211
8,0,0.286202,14.436,5.7461,2.14684,0.766303,0.266166,0.112409,0.150164,0.518728,1.8655,5.55657,10.1592,15.0417,19.9957,24.9453,29.8275,34.5225,38.7671,42.3087,50.4149
9,0,0.211108,39.3959,34.7837,29.7922,24.7926,19.7931,14.7976,9.83011,5.02198,0.986884,0.224611,0.0734452,0.0832883,0.233883,0.71572,2.07895,5.69413,14.7779,36.5313,50.2079


In [4]:
import numpy as np
import pandas as pd

# american put option
k=50; r=0.10; t=5/12; sigma=0.4; smax=100; q=0
m=20; n=10
ds = smax/m
dt = t/n
matdim=m-2
 
f = np.zeros((n+1, m+1))
matA = np.zeros((matdim+1, matdim+1))
matb = np.zeros((matdim+1, 1))
A = np.zeros((matdim+1, 1))
B = np.zeros((matdim+1, 1))
C = np.zeros((matdim+1, 1))
 
for j in range(0, m+1):
    f[n, j] = max(k - j * ds, 0)
for i in range(0, n+1):
    f[i, 0] = k
for i in range(0, n+1):
    f[i, m] = 0
 
for j in range(m-1, 1-1, -1):
    A[j-1] = 0.5 * dt * ((r-q) * j - (sigma ** 2) * (j ** 2))
    B[j-1] = 1 + ((sigma * j) ** 2) * dt + r * dt
    C[j-1] = 0.5 * dt * ((-1) * (r-q) * j - (sigma ** 2) * (j ** 2))
 
for i in range(n-1, 0-1, -1):
    mm = matdim
    for ii in range(0, matdim+1):
        for jj in range(0, matdim+1):
            if ii == jj +1:
                matA[ii, jj] = C[mm]
            elif ii == jj:
                matA[ii, jj] = B[mm]
            elif ii+1 == jj:
                matA[ii, jj] = A[mm]
            else:
                matA[ii, jj] = 0
        mm=mm-1
 
    matb[0] = f[i+1, m-1] - C[matdim]*f[i,m]
    matb[matdim] = f[i+1, 1] - A[0]*f[i,0]
 
    for jj in range(matdim-1, 1-1, -1):
        matb[jj] = f[i+1, jj+1]
 
    # LU분해가 소수점이 들어간 작은 수일 경우 pivoting을 안해주면 오차가 크게 나는 것으로 보임
    matx = Solve_SE(matA, matb)
    
    jj=m-1
    for ii in range(len(matx)):
        f[i, jj]=max(matx[ii], k - jj * ds)
        # 유러피언 옵션이면 matb[jj] = f[i+1, jj+1]
        jj=jj-1
 
for j in range(m+1):
    print("stock price: ", j*ds, "option price: ", round(f[0, j], 2))


df = pd.DataFrame(index=[x for x in range(n+1)],
                  columns=[y for y in range(m+1)])
for i in range(n, 0-1, -1):
    for j in range(m, 0-1, -1):
        df.loc[i,j] = f[i,j]

df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
name = 'put option'
df.to_excel('implicit FDM of american option.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

[[8.67643552e+00]
 [2.54297534e+01]
 [2.95137278e+01]
 [2.81295207e+01]
 [2.44946386e+01]
 [1.99825663e+01]
 [1.51542689e+01]
 [1.02436943e+01]
 [5.42289461e+00]
 [1.19233377e+00]
 [2.29063990e-01]
 [3.75188168e-02]
 [5.07689473e-03]
 [5.46406163e-04]
 [6.63421669e-05]
 [3.62743442e-04]
 [8.86347569e-03]
 [3.96592250e-01]
 [4.45816800e+01]]
[[11.62758088]
 [26.62367772]
 [29.97403543]
 [28.29834295]
 [24.55501618]
 [20.00942274]
 [15.18767824]
 [10.34687996]
 [ 5.83125102]
 [ 3.02054626]
 [ 5.72563481]
 [10.13435654]
 [14.94907559]
 [19.7471482 ]
 [24.23650893]
 [27.85342289]
 [29.25344852]
 [25.56338592]
 [44.71653917]]
[[12.63145823]
 [27.03004761]
 [30.13133916]
 [28.35779772]
 [24.58147472]
 [20.03698108]
 [15.26077535]
 [10.60699529]
 [ 6.57771099]
 [ 4.37486555]
 [ 6.27046062]
 [10.30106597]
 [15.00011516]
 [19.77978957]
 [24.30155219]
 [28.02782946]
 [29.71796789]
 [26.72954022]
 [44.72278814]]
[[12.9730046 ]
 [27.16846255]
 [30.18533424]
 [28.3793607 ]
 [24.59437473]
 [20.05848

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,50,45,40,35,30,25,20.154,16.0863,12.6494,10.3219,9.53773,10.4632,12.8803,16.3433,20.3984,24.7067,28.4268,30.2264,27.2449,13.1511,0
1,50,45,40,35,30,25,20.074,15.8742,12.285,9.82851,9.00466,10.0065,12.5615,16.1597,20.3156,24.6801,28.4176,30.223,27.2434,13.15,0
2,50,45,40,35,30,25,20.0154,15.7028,11.9592,9.34034,8.4299,9.49015,12.1953,15.9542,20.2469,24.658,28.4099,30.22,27.2417,13.148,0
3,50,45,40,35,30,25,20,15.502,11.5787,8.77996,7.805,8.98414,11.8752,15.7928,20.1987,24.6424,28.4043,30.2173,27.239,13.143,0
4,50,45,40,35,30,25,20,15.3581,11.2617,8.23784,7.11524,8.39486,11.4991,15.6034,20.1421,24.624,28.3973,30.2129,27.2327,13.1292,0
5,50,45,40,35,30,25,20,15.1897,10.8868,7.59343,6.34087,7.83658,11.1984,15.4755,20.1037,24.6109,28.3909,30.2052,27.2161,13.0894,0
6,50,45,40,35,30,25,20,15.1009,10.6148,7.00524,5.44614,7.15929,10.8434,15.3263,20.0585,24.5944,28.3794,30.1853,27.1685,12.973,0
7,50,45,40,35,30,25,20,15.0001,10.3011,6.27046,4.37487,6.57771,10.607,15.2608,20.037,24.5815,28.3578,30.1313,27.03,12.6315,0
8,50,45,40,35,30,25,20,15,10.1344,5.72563,3.02055,5.83125,10.3469,15.1877,20.0094,24.555,28.2983,29.974,26.6237,11.6276,0
9,50,45,40,35,30,25,20,15,10,5,1.19233,5.42289,10.2437,15.1543,19.9826,24.4946,28.1295,29.5137,25.4298,8.67644,0
